In [ ]:
import pickle as pkl
import os

import numpy as np
from math import floor, ceil

import matplotlib.pyplot as plt

# Load Data

In [ ]:
# Cell for loading a single SNR
directory = 'INSERT DIRECTORY' # Directory to pkl file generated by Simulation.py
fname = os.path.join(directory, 'INSERT FILENAME') # pkl Filename generated by Simulation.py
f = open(fname, 'rb')
data = pkl.load(f)
f.close()

traces = data[0]
labels = data[1]
evt_idxs = data[2]

In [ ]:
# Cell for loading multiple SNRs
fnames = [] # List of pkl Filenames generated by Simulation.py
SNRs = [] # List of corresponding SNRs as Integers

datas = []
for fname in fnames:
    f = open(os.path.join(directory, fname), 'rb')
    datas.append(pkl.load(f))
    f.close()
    
data_dict = dict(zip(SNRs, datas))

# Analyse Data

## Example Events

In [ ]:
trace1 = traces[0]
event1_pos = evt_idxs[0, 0, :]
print(event1_pos[0])

duration = event1_pos[1] - event1_pos[0]
pad = 1000
full_len = duration + (pad * 2)

full_x = np.linspace(0, full_len, full_len)
evt_x = np.linspace(pad, duration + pad, duration)

fig, ax = plt.subplots()
ax.plot(full_x, trace1[event1_pos[0]-pad : event1_pos[1]+pad])
ax.plot(evt_x, trace1[event1_pos[0] : event1_pos[1]], color='red')


## Current Histogram

In [ ]:
trace1 = traces[0]
labels1 = labels[0]

base_currents = trace1[labels1 == 0]
evt_currents = trace1[labels1 == 1]

bin_edges = np.histogram_bin_edges(trace1, bins=100)
bin_mids = bin_edges[1:] - ((bin_edges[1] - bin_edges[0]) / 2)

base_counts, _ = np.histogram(base_currents, bins=bin_edges)
evt_counts, _ = np.histogram(evt_currents, bins=bin_edges)

base_counts_norm = base_counts / len(base_currents)
evt_counts_norm = evt_counts / len(evt_currents)

In [ ]:
# Plot histogram
fig, ax = plt.subplots()
h1 = ax.bar(bin_mids, base_counts, width=np.diff(bin_edges))
h2 = ax.bar(bin_mids, evt_counts, width=np.diff(bin_edges))
h1.set_label("base")
h2.set_label("event")
ax.legend(handles=[h1, h2])

In [ ]:
# Plot normalised histogram
fig, ax = plt.subplots()

h1 = ax.bar(bin_mids, base_counts_norm, width=np.diff(bin_edges))
h2 = ax.bar(bin_mids, evt_counts_norm, width=np.diff(bin_edges))
h1.set_label("base_norm")
h2.set_label("event_norm")
ax.legend(handles=[h1, h2])

## Event Duration Histogram

In [ ]:
evt_durs = []
for group in evt_idxs:
    for evt in group:
        start = evt[0]
        end = evt[1]
        dur = end - start
        evt_durs.append(dur)

evt_durs = np.array(evt_durs)

fig, ax = plt.subplots()
_ = ax.hist(evt_durs, bins=10)

## Figure
Generate figure demonstrating the difference between events with different SNRs

In [ ]:
def plot_evt(trace, evt_pos, window_size, pad, ax, color='red'):
    '''
    Plot an example event
    '''
    
    start, end = evt_pos[0], evt_pos[1]
    dur = end - start
    
    x = np.linspace(0, window_size, window_size) / 1000000 * 1000
    
    ax.plot(x, trace[start-floor(pad) : end+ceil(pad)])
    ax.plot(x[floor(pad) : -ceil(pad)], trace[start : end], color=color)
    
def all_plot_evt(traces, evts, pad, axs, colors=['red'] * 3):
    '''
    Plot comparison of events and baseline
    '''
    assert len(evts) == 3
    longest_len = 0
    for evt in evts:
        length = evt[1] - evt[0]
        if length > longest_len:
            longest_len = length
    window_size = longest_len + (2 * pad)
    
    for i, evt in enumerate(evts):
        this_len = evt[1] - evt[0]
        this_pad = (window_size - this_len) / 2
        plot_evt(traces[i], evt, window_size, this_pad, axs[i], color=colors[i])
    
def plot_dists(data_dict, SNRs, colors, ax):
    '''
    Plot distributions of points within events of differing SNRs and baseline
    '''
    values = []
    for snr in SNRs:
        data = data_dict[snr]
        traces = data[0]
        labels = data[1]
        if snr == 0:
            labels = -1 * (labels - 1) 
        this_vals = traces[0][labels[0] == 1]
        values.append(this_vals * 1000)
        
    values_flat = np.array([val for vals in values for val in vals])
    bin_edges = np.histogram_bin_edges(values_flat, bins=100)
    bin_mids = bin_edges[1:] - np.diff(bin_edges)
    
    values_dict = dict(zip(SNRs, values))
    for i, snr in enumerate(SNRs):
        this_vals = values_dict[snr]
        counts, _ = np.histogram(this_vals, bins=bin_edges)
        
        ax.bar(bin_mids, counts / sum(counts), width=np.diff(bin_edges), color=colors[i], alpha=0.7)
        
        

In [ ]:
fig = plt.figure(figsize=(6, 4), dpi=600)
spec = fig.add_gridspec(3, 3)
ax1 = fig.add_subplot(spec[0, 0])
ax2 = fig.add_subplot(spec[1, 0])
ax3 = fig.add_subplot(spec[2, 0])
ax = fig.add_subplot(spec[:, 1:])

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

evts = [data_dict[0][2][0, 0, :], data_dict[2][2][0, 0, :], data_dict[5][2][0, 0, :]]
traces = [data_dict[0][0][0] * 1000, data_dict[2][0][0] * 1000, data_dict[5][0][0] * 1000]
axs = [ax1, ax2, ax3]
# Plot the two example events and baseline
all_plot_evt(traces, evts, 1000, axs, colors=colors)
# Plot a histogram of points within events comparing each SNR and baseline
plot_dists(data_dict, SNRs, colors, ax)

ax.set_xlabel('Current / pA', weight='bold')
ax.set_ylabel('Normalised Counts', weight='bold')

ax1.set(ylim=(-9, 9))
ax1.set_ylabel('Current / pA', weight='bold')
ax2.set(ylim=(-9, 9))
ax2.set_ylabel('Current / pA', weight='bold')
ax3.set(ylim=(-9, 9))
ax3.set_ylabel('Current / pA', weight='bold')
ax3.set_xlabel('Time / ms', weight='bold')

ax1.set(xticklabels=[])
ax2.set(xticklabels=[])

ax1.annotate('(a)', xy=(1.43, 0), xytext=(-1, 13.25), weight='bold')
ax.annotate('(b)', xy=(0, 0), xytext=(-8, 0.0644), weight='bold')

ax1.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax1.spines[axis].set_linewidth(1.5)

ax2.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax2.spines[axis].set_linewidth(1.5)

ax3.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax3.spines[axis].set_linewidth(1.5)

ax.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax.spines[axis].set_linewidth(1.5)

fig.tight_layout()

# fig.savefig('AnomSim.png')